In [1]:
# import package
# installed via pip
from emtracks.particle import trajectory_solver # main solver object
from emtracks.conversions import one_gev_c2_to_kg # conversion for q factor (transverse momentum estimate)
from emtracks.tools import *#InitConds # initial conditions namedtuple
from emtracks.mapinterp import get_df_interp_func  # factory function for creating Mu2e DS interpolation function
from emtracks.Bdist import get_B_df_distorted
import matplotlib.animation as animation
import numpy as np
from scipy.constants import c, elementary_charge
import pandas as pd
import pickle as pkl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [24,16] # bigger figures
from matplotlib import style
style.use('fivethirtyeight')
import os

testdir = "/home/darren/Desktop/plots/"
datadir = "/home/shared_data/"
plotdir = datadir+"plots/randomphi/"
mapdir = datadir+"Bmaps/"
date = "/6-20/"
newdir = datadir+'test2/'

ERROR! Please set $EMTRACKS_DDIR and $EMTRACKS_PDIR. Setting defaults (current directory)


In [2]:
def find_track_at_z(df, z): 
    delta = (df.z.max() - df.z.min()) / len(df.z)
    #delta = 10/4001   #approximate z range divided by number of points
    mask = (df['z'] < z + delta) & (df['z'] > z - delta)
    
    while (len(df.z[mask]) > 2):
        delta = delta / 2
        mask = (df.z < z + delta) & (df.z > z - delta)
    while (len(df.z[mask]) == 0):
        delta = delta*2
        mask = (df.z < z + delta) & (df.z > z - delta)
    
    if len(df.z[mask]) == 1:
        df2 = df.loc[mask]
        df2 = df2.apply(pd.to_numeric)
        return ([df2.iloc[0]['x'], df2.iloc[0]['y'], df2.iloc[0]['z'], df2.iloc[0]['t'], df2.iloc[0]['r']])

    if len(df.z[mask]) == 2:
        df2 = df.loc[mask]
        df2 = df2.apply(pd.to_numeric)

        x1 = df2.iloc[0]['x']
        x2 = df2.iloc[1]['x']
        y1 = df2.iloc[0]['y']
        y2 = df2.iloc[1]['y']
        t1 = df2.iloc[0]['t']
        t2 = df2.iloc[1]['t']
        z1 = df2.iloc[0]['z']
        z2 = df2.iloc[1]['z']

        xslope = (x2-x1)/(z2-z1)
        yslope = (y2-y1)/(z2-z1)
        tslope = (t2-t1)/(z2-z1)

        xinterp = x2 - ((z2-z)*(xslope))
        yinterp = y2 - ((z2-z)*(yslope))
        tinterp = t2 - ((z2-z)*(tslope))
        rinterp = (xinterp**2 + yinterp**2)**(1/2)

        return (xinterp, yinterp, z, tinterp, rinterp)

In [3]:
def readpkl():
    files = sorted(os.listdir(newdir))

    fieldnom = []
    fielddis = []

    thetanom = []
    thetadis = []

    phinom = []
    phidis = []

    nomdata = []
    disdata = []

    for file in files:
        if file.endswith('nom.pkl'):
            e_solvernom = trajectory_solver.from_pickle(newdir+file)
            fieldnom.append(file.split('_')[0])
            thetanom.append(e_solvernom.init_conds.theta0)
            phinom.append(e_solvernom.init_conds.phi0)
            nomdata.append(e_solvernom.dataframe)

        if file.endswith('dis.pkl'):
            e_solverdis = trajectory_solver.from_pickle(newdir+file)
            fielddis.append(file.split('_')[0])
            thetadis.append(e_solverdis.init_conds.theta0)
            phidis.append(e_solverdis.init_conds.phi0)
            disdata.append(e_solverdis.dataframe)
            
    if fieldnom == fielddis:
        if thetanom == thetadis:
            if phinom == phidis:
                field = fieldnom
                theta = thetanom
                phi = phinom
                return [field, theta, phi, nomdata, disdata]
    
    return ('something is wrong')

In [4]:
field, theta, phi, nomdata, disdata = readpkl()

In [5]:
def SeparateB(field, theta, phi, nomdata, disdata):
    uniquevals = np.unique(field)
    x = []
    for i in range(0, len(uniquevals), 1): #loop through all the unique field vals
        templist = []
        for j in range(0, len(field), 1): #separate all info into categories with same fieldval
            if field[j] == uniquevals[i]:
                templist.append([theta[j], phi[j], nomdata[j], disdata[j]])
        x.append(templist)  
        
    return [uniquevals, x]

In [1]:
def Separate2B(x, field, theta, phi, nomdata, disdata):
    uniquetheta = np.unique(theta)
    uniquephi = np.unique(phi)
    y = []
    
    for i in range(0, len(x[1]), 1): #loop through all unique vals
        templist1 = []
        for k in range(0, len(uniquetheta), 1): #0-N (theta)
            templist2 = []
            for j in range(0, len(x[1][i]), 1): 
                if uniquetheta[k] == x[1][i][j][0]:
                    templist2.append([x[0][i], theta[j], phi[j], x[1][i][2], x[1][i][3]])
            templist1.append(templist2)
        y.append(templist1)
    
    return y

In [7]:
def SeparateTheta(field, theta, phi, nomdata, disdata):
    uniquevals = np.unique(theta)
    x = []
    for i in range(0, len(uniquevals), 1): #loop through all the unique field vals
        templist = []
        for j in range(0, len(theta), 1): #separate all info into categories with same fieldval
            if theta[j] == uniquevals[i]:
                templist.append([field[j], phi[j], nomdata[j], disdata[j]])
        x.append(templist)  
        
    return [uniquevals, x]

In [8]:
def SeparatePhi(field, theta, phi, nomdata, disdata):
    uniquevals = np.unique(phi)
    x = []
    for i in range(0, len(uniquevals), 1): #loop through all the unique field vals
        templist = []
        for j in range(0, len(phi), 1): #separate all info into categories with same fieldval
            if phi[j] == uniquevals[i]:
                templist.append([field[j], phi[j], nomdata[j], disdata[j]])
        x.append(templist)  
    
    return [uniquevals, x]

In [9]:
x = SeparateB(field, theta, phi, nomdata, disdata)
y = SeparatePhi(field, theta, phi, nomdata, disdata)
z = SeparateTheta(field, theta, phi, nomdata, disdata)

In [27]:
x[0][1]

'5.0'

In [39]:
e = Separate2B(x, field, theta, phi, nomdata, disdata)

In [40]:
len(e[0][0][0]) #fieldval, thetaval, unique phi val

5

In [41]:
len(e[0][0]) #5 different phi vals and their unique data

5